<a href="https://colab.research.google.com/github/lebe1/text-oriented-data-science-project/blob/main/Data_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Dataset



## Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
folder_path = '/content/drive/MyDrive/'

## Imports

In [3]:
!pip install wandb

In [4]:
import pandas as pd
import numpy as np
import nltk
import time
from nltk.corpus import stopwords
import string
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report

import wandb

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Reading the CSV File

In [5]:
file_name = 'combined_reviews.csv'
file_path = folder_path + file_name

#df = pd.read_csv(file_path)

csv_path = '/content/drive/MyDrive/DOPP_Ex2_data/reviews_train_long.csv'
df = pd.read_csv(csv_path)

In [6]:
df.head()

,rating,text_type,reviewText
0,4.0,reviewText,Very comfortable shoes.
1,4.0,reviewText,I really like this cleanser. It is gentle on t...
2,5.0,reviewText,I do my own nails at home and love CND Shellac...
3,5.0,reviewText,I do my own nails at home and love CND Shellac...
4,5.0,reviewText,I do my own nails at home and love CND Shellac...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70466 entries, 0 to 70465
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rating      70466 non-null  float64
 1   text_type   70466 non-null  object 
 2   reviewText  70466 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.6+ MB


# Build the model

In [8]:
# Preprocessing
def preprocess_text(text):
    # Convert only string instances to lowercase
    text = text.lower() if isinstance(text, str)  else ''
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = text.split()
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)


df['preprocessedText'] = df['reviewText'].apply(preprocess_text)

# Tokenize text again for word2vec
df['tokenized_text'] = df['preprocessedText'].str.split()

df['preprocessedText'].head()


,preprocessedText
0,comfortable shoes
1,really like cleanser gentle skin face feels cl...
2,nails home love cnd shellac great prodct truly...
3,nails home love cnd shellac great prodct truly...
4,nails home love cnd shellac great prodct truly...


In [9]:
y = df['rating']
X = pd.DataFrame({'preprocessedText': df['preprocessedText'], 'tokenized_text': df['tokenized_text'], 'reviewText': df["reviewText"]})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine train and test sets
df_train = pd.DataFrame({'preprocessedText': X_train['preprocessedText'], 'tokenized_text': X_train['tokenized_text'], 'reviewText': X_train["reviewText"], 'rating': y_train})
df_test = pd.DataFrame({'preprocessedText': X_test['preprocessedText'], 'tokenized_text': X_test['tokenized_text'], 'reviewText': X_test["reviewText"], 'rating': y_test})

# Save them into csv files
df_train.to_csv(folder_path + 'train.csv', index=True)
df_test.to_csv(folder_path + 'test.csv', index=True)


In [10]:
X_train.head()

,preprocessedText,tokenized_text,reviewText
53828,love look feel shoes style weightlifting super...,"[love, look, feel, shoes, style, weightlifting...",I LOVE the look and feel of these shoes for st...
36658,dry skin leaves clean fresh,"[dry, skin, leaves, clean, fresh]",Does not dry out the skin. Leaves it clean and...
35870,older generation shoe survived 2 3 races mud p...,"[older, generation, shoe, survived, 2, 3, race...",I had the older generation of this shoe that s...
47535,love shoes circuit training,"[love, shoes, circuit, training]",I love these shoes for circuit training.
15347,purchased daughter gym class loves cute comfor...,"[purchased, daughter, gym, class, loves, cute,...",Purchased for my daughter for gym class. She ...


In [11]:
print(common_texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [12]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer()
X_tfidf_train = tfidf.fit_transform(X_train['preprocessedText']).toarray()
X_tfidf_test = tfidf.transform(X_test['preprocessedText']).toarray()

# Word2Vec Embeddings
w2v_model_train = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_embedding(word_list, model):
    word_vecs = [model.wv[word] for word in word_list if word in model.wv]
    if word_vecs:
        return np.mean(word_vecs, axis=0)
    else:
        return np.zeros(model.vector_size)

X_w2v_train = np.array([get_sentence_embedding(word_list, w2v_model_train) for word_list in X_train['tokenized_text']])
X_w2v_test = np.array([get_sentence_embedding(word_list, w2v_model_train) for word_list in X_test['tokenized_text']])

X_train_vectorized = np.hstack((X_tfidf_train, X_w2v_train))
X_test_vectorized = np.hstack((X_tfidf_test, X_w2v_test))


## Including wandb for analysis during model training

In [13]:
wandb.login()

wandb: Currently logged in as: l-beccard (l-beccard-tu-wien). Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb_project_name = "DOPP analysis"
wandb_run_name = "rf_experiment-9-estimators-100"

rf_config = {
    "n_estimators": 100,
    "max_depth": None,
    "random_state": 42,
    "test_size": 0.2,
    "dataset": "Word2Vec"
}


wandb.init(
    project=wandb_project_name,
    name=wandb_run_name,
    config=rf_config
)


start_time = time.time()

rf_model = RandomForestClassifier(
    n_estimators=rf_config["n_estimators"],
    max_depth=rf_config["max_depth"],
    random_state=rf_config["random_state"]
)
rf_model.fit(X_train_vectorized, y_train)

y_pred_rf = rf_model.predict(X_test_vectorized)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_rf, average='macro')
precision = precision_score(y_test, y_pred_rf, average='macro')
recall = recall_score(y_test, y_pred_rf, average='macro')

# Log metrics to W&B
wandb.log({
    "Execution Time": execution_time,
    "F1 Score": f1Score,
    "Precision Score": precision,
    "Recall Score": recall
})

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_rf))

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
svc_config = {
    "random_state": 42,
    "test_size": 0.2,
    "max_iter": 1000,
    "penalty": "l1",
    "dataset": "Combined"
}

wandb_project_name = "DOPP analysis"

wandb.init(
    project=wandb_project_name,
    name="svc_experiment-4",
    config=svc_config
)

start_time = time.time()

linear_svc_model = LinearSVC(random_state=svc_config["random_state"], penalty=svc_config["penalty"])
linear_svc_model.fit(X_train_vectorized, y_train)

y_pred_svc = linear_svc_model.predict(X_test_vectorized)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_svc, average='macro')
precision = precision_score(y_test, y_pred_svc, average='macro')
recall = recall_score(y_test, y_pred_svc, average='macro')

wandb.log({
    "Execution Time": execution_time,
    "F1 Score": f1Score,
    "Precision Score": precision,
    "Recall Score": recall
})

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_svc))

wandb.finish()

## Qualitative misclassification analysis

In [ ]:
print(np.unique(y_pred_svc))
print(np.unique(y_pred_rf))


We see our models predict all given classes.  
Now, let's understand why some classes are misclassified.

In [ ]:
false_preds_svc = y_pred_svc != y_test

misclassified_predictions = y_pred_svc[false_preds_svc]
misclassified_labels = y_test[false_preds_svc]

In [ ]:
misclassified_predictions

In [ ]:
np.unique(misclassified_predictions, return_counts=True)

By this frequency count, it is observable that most of the time a 5-star-rating is predicting wrong, which makes sense since the original dataset is quite imbalanced. The grade 2 has been misclassified the least but is also the least represented class in the dataset.

In [ ]:
misclassified_labels

In [ ]:
df_misclassified = df.iloc[misclassified_labels.index]

In [ ]:
df_misclassified["misclassified_rating"] = misclassified_predictions

In [ ]:
df_misclassified.iloc[0]

In [ ]:
df_misclassified.iloc[0]["reviewText"]

Based on the review text, we can observe that the model does not really understand the final critizing words of the this review. The review text itself is reasonable to give this three stars.

In [ ]:
df_misclassified.iloc[14]

In [ ]:
df_misclassified.iloc[14]["reviewText"]

Again, the review text presents some kind of critique, which should be understood by the model not to rate it with five stars.

In [ ]:
df_misclassified.iloc[31]

In [ ]:
df_misclassified.iloc[31]["reviewText"]

This is an interesting case since we have the opposite behaviour of the model now predicting a higher rated product of 4 with a lower rating of 2.
This review text is easy to understand for a human but since we remove stopwords for model training it might be possible that the sentence ends up with complete different meaning with meaningful words like `died` and `carefully`. Based on an assumption like this a two star rating seems plausible.

## Balancing optimizations

In [ ]:
df_train["rating"].value_counts()

We have only 306 two star ratings as the least represented class. Therefore, we will take only 306 random samples from the other classes.

In [ ]:
df_rating_2 = df_train[df_train["rating"] == 2]

In [ ]:
df_rating_1 = df_train[df_train["rating"] == 1].sample(n=306, random_state=42)
df_rating_3 = df_train[df_train["rating"] == 3].sample(n=306, random_state=42)
df_rating_4 = df_train[df_train["rating"] == 4].sample(n=306, random_state=42)
df_rating_5 = df_train[df_train["rating"] == 5].sample(n=306, random_state=42)

In [ ]:
# Merge all dataframes
df_balanced = pd.concat([df_rating_2, df_rating_1, df_rating_3, df_rating_4, df_rating_5])

In [ ]:
df_balanced.shape

In [ ]:
df_balanced["rating"].value_counts()

In [ ]:
df_balanced.head()

In [ ]:
y_train_balanced = df_balanced['rating']

X_tfidf_train_balanced = tfidf.fit_transform(df_balanced['preprocessedText']).toarray()
X_tfidf_test_balanced = tfidf.transform(X_test['preprocessedText']).toarray()

# Using w2v model with train set only as recommended here: https://stackoverflow.com/a/70900433/19932351
X_w2v_train_balanced = np.array([get_sentence_embedding(word_list, w2v_model_train) for word_list in df_balanced['tokenized_text']])
X_w2v_test_balanced = np.array([get_sentence_embedding(word_list, w2v_model_train) for word_list in X_test['tokenized_text']])

X_train_vectorized_balanced = np.hstack((X_tfidf_train_balanced, X_w2v_train_balanced))
X_test_vectorized_balanced = np.hstack((X_tfidf_test_balanced, X_w2v_test_balanced))

In [ ]:
X_train_vectorized_balanced.shape

In [ ]:
X_test_vectorized_balanced.shape

In [ ]:
wandb_project_name = "DOPP analysis"
wandb_run_name = "rf_balanced-2-estimators-100"

rf_config = {
    "n_estimators": 100,
    "max_depth": None,
    "random_state": 42,
    "test_size": 0.2,
    "dataset": "Balanced-Combined"
}


wandb.init(
    project=wandb_project_name,
    name=wandb_run_name,
    config=rf_config
)


start_time = time.time()

rf_model = RandomForestClassifier(
    n_estimators=rf_config["n_estimators"],
    max_depth=rf_config["max_depth"],
    random_state=rf_config["random_state"]
)
rf_model.fit(X_train_vectorized_balanced, y_train_balanced)

y_pred_rf = rf_model.predict(X_test_vectorized_balanced)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_rf, average='macro')
precision = precision_score(y_test, y_pred_rf, average='macro')
recall = recall_score(y_test, y_pred_rf, average='macro')

# Log metrics to W&B
wandb.log({
    "Execution Time": execution_time,
    "F1 Score": f1Score,
    "Precision Score": precision,
    "Recall Score": recall
})

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_rf))

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
svc_config = {
    "random_state": 42,
    "test_size": 0.2,
    "max_iter": 1000,
    "penalty": "l2",
    "dataset": "Balanced-Combined"
}

wandb_project_name = "DOPP analysis"

wandb.init(
    project=wandb_project_name,
    name="svc_balanced",
    config=svc_config
)

start_time = time.time()

linear_svc_model = LinearSVC(random_state=svc_config["random_state"], penalty=svc_config["penalty"])
linear_svc_model.fit(X_train_vectorized_balanced, y_train_balanced)

y_pred_svc = linear_svc_model.predict(X_test_vectorized_balanced)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_svc, average='macro')
precision = precision_score(y_test, y_pred_svc, average='macro')
recall = recall_score(y_test, y_pred_svc, average='macro')

wandb.log({
    "Execution Time": execution_time,
    "F1 Score": f1Score,
    "Precision Score": precision,
    "Recall Score": recall
})

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_svc))

wandb.finish()

By the metrices, we can observe that the random forest improves in terms of predicting several classes, whereas the svc model trained with a balanced dataset results in a lower performance not recognizing the majority class that well anymore.

## Oversampling


In [ ]:
df_rating_1 = df_train[df_train["rating"] == 1]
df_rating_2 = df_train[df_train["rating"] == 2]
df_rating_3 = df_train[df_train["rating"] == 3]
df_rating_4 = df_train[df_train["rating"] == 4]
df_rating_5 = df_train[df_train["rating"] == 5]
print("1",len(df_rating_1))
print("2", len(df_rating_2))
print("3", len(df_rating_3))
print("4",len(df_rating_4))
print("5", len(df_rating_5))

In our next approach, we target the sample count of 1414, which is the count of class 4, the second most represented class. Therefore, we have to oversample classes 1 to 3 and undersample the majority class 5.

In [ ]:
df_rating_1 = df_train[df_train["rating"] == 1].sample(n=1414, random_state=42, replace=True)
df_rating_2 = df_train[df_train["rating"] == 2].sample(n=1414, random_state=42, replace=True)
df_rating_3 = df_train[df_train["rating"] == 3].sample(n=1414, random_state=42, replace=True)
df_rating_5 = df_train[df_train["rating"] == 5].sample(n=1414, random_state=42)

In [ ]:
df_oversampled = pd.concat([df_rating_2, df_rating_1, df_rating_3, df_rating_4, df_rating_5])

In [ ]:
df_oversampled["rating"].value_counts()

In [ ]:
y_train_oversampled = df_oversampled['rating']

X_tfidf_train_oversampled = tfidf.fit_transform(df_oversampled['preprocessedText']).toarray()
X_tfidf_test_oversampled = tfidf.transform(X_test['preprocessedText']).toarray()

# Using w2v model with train set only as recommended here: https://stackoverflow.com/a/70900433/19932351
X_w2v_train_oversampled = np.array([get_sentence_embedding(word_list, w2v_model_train) for word_list in df_oversampled['tokenized_text']])
X_w2v_test_oversampled = np.array([get_sentence_embedding(word_list, w2v_model_train) for word_list in X_test['tokenized_text']])

X_train_vectorized_oversampled = np.hstack((X_tfidf_train_oversampled, X_w2v_train_oversampled))
X_test_vectorized_oversampled = np.hstack((X_tfidf_test_oversampled, X_w2v_test_oversampled))


In [ ]:
wandb_project_name = "DOPP analysis"
wandb_run_name = "rf_oversampled"

rf_config = {
    "n_estimators": 100,
    "max_depth": None,
    "random_state": 42,
    "test_size": 0.2,
    "dataset": "Oversampled-Combined"
}


wandb.init(
    project=wandb_project_name,
    name=wandb_run_name,
    config=rf_config
)


start_time = time.time()

rf_model = RandomForestClassifier(
    n_estimators=rf_config["n_estimators"],
    max_depth=rf_config["max_depth"],
    random_state=rf_config["random_state"]
)
rf_model.fit(X_train_vectorized_oversampled, y_train_oversampled)

y_pred_rf = rf_model.predict(X_test_vectorized_oversampled)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_rf, average='macro')
precision = precision_score(y_test, y_pred_rf, average='macro')
recall = recall_score(y_test, y_pred_rf, average='macro')

# Log metrics to W&B
wandb.log({
    "Execution Time": execution_time,
    "F1 Score": f1Score,
    "Precision Score": precision,
    "Recall Score": recall
})

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_rf))

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
svc_config = {
    "random_state": 42,
    "test_size": 0.2,
    "max_iter": 1000,
    "penalty": "l2",
    "dataset": "Oversampled-Combined"
}

wandb_project_name = "DOPP analysis"

wandb.init(
    project=wandb_project_name,
    name="svc_balanced",
    config=svc_config
)

start_time = time.time()

linear_svc_model = LinearSVC(random_state=svc_config["random_state"], penalty=svc_config["penalty"])
linear_svc_model.fit(X_train_vectorized_oversampled, y_train_oversampled)

y_pred_svc = linear_svc_model.predict(X_test_vectorized_oversampled)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_svc, average='macro')
precision = precision_score(y_test, y_pred_svc, average='macro')
recall = recall_score(y_test, y_pred_svc, average='macro')

wandb.log({
    "Execution Time": execution_time,
    "F1 Score": f1Score,
    "Precision Score": precision,
    "Recall Score": recall
})

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_svc))

wandb.finish()

## Data Augmentation

In [11]:
def get_sentence_embedding(word_list, model):
    word_vecs = [model.wv[word] for word in word_list if word in model.wv]
    if word_vecs:
        return np.mean(word_vecs, axis=0)
    else:
        return np.zeros(model.vector_size)

In [12]:
csv_path = '/content/drive/MyDrive/DOPP_Ex2_data/reviews_train_long.csv'
df_da = pd.read_csv(csv_path)

In [13]:
df_da = df_da.sample(frac=1).reset_index(drop=True)
df_da.head(2)

,rating,text_type,reviewText
0,5.0,reviewText_german_french_english,Great product
1,5.0,reviewText_german_french_english,I love these shoes. I train with it and take i...


In [14]:
df_da['preprocessedText'] = df_da['reviewText'].apply(preprocess_text)
df_da['tokenized_text'] = df_da['preprocessedText'].str.split()

In [ ]:
y_train_da = df_da['rating']

tfidf = TfidfVectorizer()
X_tfidf_train_da =  tfidf.fit_transform(df_da['preprocessedText']).toarray()
X_tfidf_test_da = tfidf.transform(X_test['preprocessedText']).toarray()

# Word2Vec Embeddings
w2v_model_train = Word2Vec(sentences=df_da['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)

# Using w2v model with train set only as recommended here: https://stackoverflow.com/a/70900433/19932351
X_w2v_train_da = np.array([get_sentence_embedding(word_list, w2v_model_train) for word_list in df_da['tokenized_text']])
X_w2v_test_da = np.array([get_sentence_embedding(word_list, w2v_model_train) for word_list in X_test['tokenized_text']])

X_train_vectorized_da = np.hstack((X_tfidf_train_da, X_w2v_train_da))
X_test_vectorized_da = np.hstack((X_tfidf_test_da, X_w2v_test_da))

In [ ]:
wandb_project_name = "DOPP analysis"
wandb_run_name = "rf_da"

rf_config = {
    "n_estimators": 100,
    "max_depth": None,
    "random_state": 42,
    "test_size": 0.2,
    "dataset": "Data-Augmentation"
}


wandb.init(
    project=wandb_project_name,
    name=wandb_run_name,
    config=rf_config
)


start_time = time.time()

rf_model = RandomForestClassifier(
    n_estimators=rf_config["n_estimators"],
    max_depth=rf_config["max_depth"],
    random_state=rf_config["random_state"]
)
rf_model.fit(X_train_vectorized_da, y_train_da)

y_pred_rf = rf_model.predict(X_test_vectorized_da)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_rf, average='macro')
precision = precision_score(y_test, y_pred_rf, average='macro')
recall = recall_score(y_test, y_pred_rf, average='macro')

# Log metrics to W&B
wandb.log({
    "Execution Time": execution_time,
    "F1 Score": f1Score,
    "Precision Score": precision,
    "Recall Score": recall
})

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_rf))

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
svc_config = {
    "random_state": 42,
    "test_size": 0.2,
    "max_iter": 1000,
    "penalty": "l2",
    "dataset": "Data-Augmentation"
}

wandb_project_name = "DOPP analysis"

wandb.init(
    project=wandb_project_name,
    name="svc_balanced",
    config=svc_config
)

start_time = time.time()

linear_svc_model = LinearSVC(random_state=svc_config["random_state"], penalty=svc_config["penalty"])
linear_svc_model.fit(X_train_vectorized_da, y_train_da)

y_pred_svc = linear_svc_model.predict(X_test_vectorized_da)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_svc, average='macro')
precision = precision_score(y_test, y_pred_svc, average='macro')
recall = recall_score(y_test, y_pred_svc, average='macro')

wandb.log({
    "Execution Time": execution_time,
    "F1 Score": f1Score,
    "Precision Score": precision,
    "Recall Score": recall
})

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_svc))

wandb.finish()